In [47]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from urllib.parse import urlparse
from selenium_stealth import stealth # 셀레니움 스텔스 기능

# 예외 처리
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

import time
import pandas as pd
import numpy as np
import os
import random

In [ ]:
# 사용자 지정 함수 부분 

# smartDF 만드는 함수
def makeLinkDF(storelink, storename):
    smartDF = pd.DataFrame([storelink, storename])
    smartDF.index=['링크','상점']
    smartDF = smartDF.T
    return smartDF

# 기존 파일영역(saveDF)에 추가(newDF)해줌.
def concatFile(saveDF, newDF): 
    saveDF=pd.read_excel('./Data/output.xlsx')
    saveDF.drop(['Unnamed: 0'], axis=1, inplace=True) # 불러오면 'Unnamed: 0' 제거해줘야함.
    updateDF=pd.concat([saveDF, newDF])
    return updateDF

# 스크롤을 내리는 함수 (페이지 끝까지)
def scroll_to_end(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) # 페이지가 로드되는 동안 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# 링크 DF 만드는 함수 (상품, 해당 링크)
def makeLinks(name_list, link_list):
    dataDF = pd.DataFrame([name_list, link_list])
    dataDF.index=['상품','링크']
    dataDF = dataDF.T
    return dataDF

# smartDF 파일 업데이트하기
def updateLink(smartDF, filename):
    baseDF=pd.read_excel(filename)
    smartDF=pd.concat([baseDF, smartDF])
    smartDF.drop(['Unnamed: 0'], axis=1, inplace=True) # 'Unnamed: 0' 제거해줘야함.
    smartDF.to_excel(filename)
    return smartDF

- 초기 방법 코드

In [ ]:
agent_option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=agent_option)

user_agent_string = 'Mozilla/5.0'
agent_option.add_argument('user-agent=' + user_agent_string)

original_window = driver.current_window_handle

### 사이트 진입
url = 'https://search.shopping.naver.com/catalog/38387900618?&NaPm=ct%3Dm3o6cnzk%7Cci%3Dd6e42fe5a7117073bf0cf10be64e41caf8e0dd9b%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3D57ed9b59c78d943e9049a790240b2c9b7e878a35'
driver.get(url)

# 링크 클릭
link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="section_price"]/div[4]/ul/li[1]/div/div[2]/a'))
)
link.click()

# 새 탭으로 전환
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)

# 새 탭의 URL 가져오기
current_url = driver.current_url

# 도메인 추출
parsed_url = urlparse(current_url)
domain = parsed_url.netloc

base_url = f"{parsed_url.scheme}://{parsed_url.netloc}" 

print(f"새 탭에서 열린 URL: {current_url}") # 'https://smartstore.naver.com/fullylight/products/...
print(f"도메인 주소: {domain}") # ex) smartstore.naver.com
print(f"base_url: {base_url}") # ex) https://smartstore.naver.com

# 새 탭 닫기 및 원래 탭으로 복귀
driver.close()
driver.switch_to.window(original_window)

# links = driver.find_elements(By.CLASS_NAME, "productList_title__R1qZP")
# for idx, link in enumerate(links):
#     link.click()

# time.sleep(3)
driver.quit()

KeyboardInterrupt: 

- 가격비교 1개에서 링크 가져오기 샘플

In [ ]:
agent_option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=agent_option)

user_agent_string = 'Mozilla/5.0'
agent_option.add_argument('user-agent=' + user_agent_string)

original_window = driver.current_window_handle

### 사이트 진입
url = 'https://search.shopping.naver.com/catalog/45311333618?&NaPm=ct%3Dm3ogn9t4%7Cci%3Db9b07e10143aa65bb1fa8a4472d058f5541ff320%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3D5a4815a72a9899c0f573073d053040e78a77dbe9'
driver.get(url)

# URL 가져오기
current_url = driver.current_url

# 도메인 추출
parsed_url = urlparse(current_url)
path_seg = parsed_url.path.split('/')
# 링크 생성
smartstore = []

for i in range(2, 12):
    link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, f'//*[@id="section_price"]/div[4]/ul/li[{i}]/div/div[1]')))
    link.click()
    text = link.text
    time.sleep(2)

    # 새 탭으로 전환
    new_window = [window for window in driver.window_handles if window != original_window][0]
    driver.switch_to.window(new_window)
    time.sleep(1)

    # 새 탭의 URL 가져오기
    current_url = driver.current_url

    # 도메인 추출
    parsed_url = urlparse(current_url)
    domain = parsed_url.netloc
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}" 

    # 스마트스토어 리스트 추출
    if base_url == 'https://smartstore.naver.com':
        path_seg = parsed_url.path.split('/')
        smart_id = path_seg[1]
        qna = base_url + '/' + path_seg[1] + '/qna'
        if qna not in smartstore:
            smartstore.append(qna)
        else:
            pass

    print(f"도메인 주소: {domain}") # ex) smartstore.naver.com
    print(f"base_url: {base_url}") # ex) https://smartstore.naver.com
    print(f"smart_id: {smart_id}") # ex) isround
    print(f"text: {text}")

    # 새 탭 닫기 및 원래 탭으로 복귀
    driver.close()
    driver.switch_to.window(original_window)
    time.sleep(2)
driver.quit()

- 상품들 링크 만들기

In [ ]:
driver = webdriver.Chrome()
agent_option = webdriver.ChromeOptions()

user_agent_string = 'Mozilla/5.0'
agent_option.add_argument('user-agent=' + user_agent_string)

url = "https://search.shopping.naver.com/search/category/100007873?adQuery&catId=50000024&origQuery&pagingIndex=1&pagingSize=80&productSet=total&query&sort=review_rel&timestamp=&viewType=list"
driver.get(url)

for i in range(1,11):

    driver.implicitly_wait(5)

    # 스크롤을 최대한 밑으로 내림 (동적 Javascript)
    scroll_to_end(driver)
    time.sleep(5)
    
    driver.find_element(By.CLASS_NAME, 'pagination_next__pZuC6').click() # 다음 클릭하기
    time.sleep(10)

    # 스크롤을 최대한 밑으로 내림 (동적 Javascript)
    scroll_to_end(driver)
    time.sleep(5)

    # 페이지 소스 soup로 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(1)

    ### 링크 가져오기
    
    link_tags = soup.select('div.product_title__Mmw2K > a')

    ### 링크 리스트
    link_list=[]
    name_list=[]
    for tag in link_tags:

        name_tag = tag.text.strip() # 이름 가져오기
        name_list.append(name_tag)

        link_tag = tag['href'] # 링크 가져오기
        link_list.append(link_tag)

    print(name_list)
    print(link_list)

    linkDF = makeLinks(name_list, link_list)
    linkDF = updateLink(linkDF)
    print(f'link 누적개수 : {len(linkDF)}')

driver.quit()

- smartlinks 사이트 형태 파악하기

In [121]:
storelinks = pd.read_excel('smartlinks.xlsx')
agent_option = webdriver.ChromeOptions()
checklist = []
for idx, storelink in enumerate(storelinks['링크']):
    ### 사이트 진입
    user_agent_string = 'Mozilla/5.0'
    agent_option.add_argument('user-agent=' + user_agent_string)
    driver = webdriver.Chrome(options=agent_option)
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    driver.get(storelink)
    time.sleep(1)

    # URL 가져오기
    current_url = driver.current_url
    main_parsed_url = urlparse(current_url)
    time.sleep(1)
    main_path_seg = main_parsed_url.path.split('/')
    print(f"main_parsed_url: {main_parsed_url}")
    print(f"main_path_seg[1]: {main_path_seg[1]}")
    if main_path_seg[1] == 'catalog':
        checklist.append('카탈로그')
    else:
        checklist.append('그 외')
    driver.quit()

# 카탈로그 라벨링한 파일 내보내기
storelinks['상점'] = checklist
storelinks.to_excel('storelinks.xlsx')

main_parsed_url: ParseResult(scheme='https', netloc='search.shopping.naver.com', path='/catalog/38117670335', params='', query='&NaPm=ct%3Dm3pzy9qg%7Cci%3Dcadcdab6002284e6dda48ccb3f868df3a577d0cd%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3D3a525bcdaa697be910eff122f39470d25b835efe', fragment='')
main_path_seg[1]: catalog
main_parsed_url: ParseResult(scheme='https', netloc='smartstore.naver.com', path='/remora/products/2024404149', params='', query='nl-query=-', fragment='')
main_path_seg[1]: remora
main_parsed_url: ParseResult(scheme='https', netloc='search.shopping.naver.com', path='/catalog/51157367618', params='', query='&NaPm=ct%3Dm3pzyqpk%7Cci%3Dd9aee3a9e30302dd2a1c6e40a3b2fac36431fd27%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3D6e8214500aca2cbd6147cb599799a5b6c79ede44', fragment='')
main_path_seg[1]: catalog
main_parsed_url: ParseResult(scheme='https', netloc='smartstore.naver.com', path='/healthy5/products/4727021213', params='', query='nl-query=-', fragment='')
main_path_seg[1]: healthy5
main_parsed_ur

In [ ]:
# 파일 변수 지정
file = './rawfile.xlsx'

# 카탈로그 파일 가져오기
catalog_links = pd.read_excel('storelinks(catalog).xlsx')

### 사이트 진입
for catalog in catalog_links['링크']:
    agent_option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=agent_option)

    user_agent_string = 'Mozilla/5.0'
    agent_option.add_argument('user-agent=' + user_agent_string)

    original_window = driver.current_window_handle
    driver.get(catalog)

    product_list = driver.find_elements(By.CLASS_NAME, 'productList_title__R1qZP')
    
    # 링크 생성 또는 링크 로딩
    if not os.path.exists(file):
        storelink, storename = [], []
    else:
        fileDF = pd.read_excel(file)
        storelink = fileDF['링크'].to_list()
        storename = fileDF['상점'].to_list()

    for i in range(1, len(product_list)+1):
        try:
            # XPATH로 차례로 접근
            link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (By.XPATH, f'//*[@id="section_price"]/div[4]/ul/li[{i}]/div/div[1]')))

            link.click()

            text = link.text
            time.sleep(2)

            # 새 탭으로 전환
            if len(driver.window_handles) > 1:
                new_window = [window for window in driver.window_handles if window != original_window][0]
                driver.switch_to.window(new_window)
                time.sleep(1)

                # 새 탭의 URL 가져오기
                current_url = driver.current_url

                # 도메인 추출
                parsed_url = urlparse(current_url)
                domain = parsed_url.netloc
                base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"

                # 스마트스토어 리스트 추출
                if base_url == 'https://smartstore.naver.com':
                    path_seg = parsed_url.path.split('/')
                    smart_id = path_seg[1]
                    qna = base_url + '/' + path_seg[1] + '/qna'

                    # storelink 안에 리스트가 없는 경우만 추가 (중복 방지)
                    if qna not in storelink:
                        storelink.append(qna)
                        storename.append(text)
                        print(f"도메인 주소: {domain}") # ex) smartstore.naver.com
                        print(f"base_url: {base_url}") # ex) https://smartstore.naver.com
                        print(f"smart_id: {smart_id}") # ex) isround
                        print(f"text: {text}")
                        print()
                    
                    else:
                        print('새 창이 안열림.')
                        print()

                # 새 탭 닫기 및 원래 탭으로 복귀
                driver.close()
                driver.switch_to.window(original_window)
                time.sleep(1)

        # 오류 발생 시 예외 처리 후 넘어감.
        except Exception as e:
            print(f'예외 오류 발생 코드: {e}')
            continue

    driver.quit()
    # smartDF = makeLinkDF(storelink, storename)
    smartDF = pd.DataFrame({'링크': storelink, '상점': storename})

    # 파일 내보내기
    smartDF.to_excel(file, index=False)

새 창이 안열림.

새 창이 안열림.

새 창이 안열림.

새 창이 안열림.

새 창이 안열림.

새 창이 안열림.

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: bioglobal
text: 너만아는과자점
네이버플러스멤버십

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: topmall001
text: top탑몰
네이버플러스멤버십

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: trendyfood
text: 트렌디푸드샵
네이버플러스멤버십

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: dh1shop
text: 동행원샵
네이버플러스멤버십

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: all-in-one_store
text: all in one 스토어
네이버플러스멤버십

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: three-daughters
text: 매일유업공식대리점
네이버플러스멤버십

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: globalbucket
text: 글로벌버킷
네이버플러스멤버십

새 창이 안열림.

새 창이 안열림.

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: dazunu
text: 다주누
네이버플러스멤버십

Exception ignored in: <function Service.__del__ at 0x0000020AF13ED9E0>
Traceback (most recent call last):
  File "c:\Users\KDP-50\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\Users\KDP-50\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\KDP-50\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\KDP-50\anaconda3\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\KDP-50\anaconda3\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\KDP-50\anaconda3\Lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handl

예외 오류 발생 코드: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.85)
Stacktrace:
	GetHandleVerifier [0x00007FF75FE36CB5+28821]
	(No symbol) [0x00007FF75FDA3840]
	(No symbol) [0x00007FF75FC4578A]
	(No symbol) [0x00007FF75FC1F4F5]
	(No symbol) [0x00007FF75FCC6247]
	(No symbol) [0x00007FF75FCDECE2]
	(No symbol) [0x00007FF75FCBF0A3]
	(No symbol) [0x00007FF75FC8A778]
	(No symbol) [0x00007FF75FC8B8E1]
	GetHandleVerifier [0x00007FF76016FCAD+3408013]
	GetHandleVerifier [0x00007FF76018741F+3504127]
	GetHandleVerifier [0x00007FF76017B5FD+3455453]
	GetHandleVerifier [0x00007FF75FEFBDBB+835995]
	(No symbol) [0x00007FF75FDAEB5F]
	(No symbol) [0x00007FF75FDAA814]
	(No symbol) [0x00007FF75FDAA9AD]
	(No symbol) [0x00007FF75FD9A199]
	BaseThreadInitThunk [0x00007FFB3675259D+29]
	RtlUserThreadStart [0x00007FFB3892AF38+40]

도메인 주소: smartstore.naver.com
base_url: https://smartstore.naver.com
smart_id: cocochu
text: 코코오톡
네이버플러스멤버십



KeyboardInterrupt: 

In [ ]:
# 파일 변수 지정
file = './rawfile.xlsx'

# 파일 불러오기
fileDF = pd.read_excel(file)

for link in fileDF['링크']:

    # 사이트 진입
    agent_option = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=agent_option)

    user_agent_string = 'Mozilla/5.0'
    agent_option.add_argument('user-agent=' + user_agent_string)

    # 링크 가져오기
    driver.get(link)
    driver.implicitly_wait(5)

    # page 개수 가져오기
    pagenum = driver.find_elements(By.CLASS_NAME, "UWN4IvaQza._nlog_click")

    # 해당 페이지 텍스트 담을 리스트 만들기
    text_list=[]
    for page in pagenum:
        page.click()
        # 해당 페이지 html 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 텍스트 가져오기
        texts = soup.find_all('div', class_='_38rS4qj1Bp')
        
        for tag in texts:
            name_tag = tag.text.strip()
            text_list.append(name_tag)
    print(text_list)
    driver.quit()

['NEW상품구매후 환불요청한 상황입니다!\n상품회수해가신지 일주일이 지났습니다.바로 ㅎ환불처리 해주세요!', '상품 표기 내역에 성분이 많이들어갈수록 앞에 표기되는걸로 알고있는데ㅜㅜ\n콜라겐보다도 정제수랑 블루베리농축액이 더 많이 \n 들어간건가요? \n다른제품들이랑 비교했을때 저거 하나가 걸려서 구매가 망설여지는대 설명 부탁드립니당', '비밀글비밀글입니다.', '비밀글비밀글입니다.', '비밀글비밀글입니다.', '어제 문의글 남겼는데 확인부탁드려요', '임산부 또는 임신 예정인 사람도 먹어도 되나요??', '비밀글비밀글입니다.', '비밀글비밀글입니다.', '7일날 주문했는데 언제발송되나요 ?', '아침 공복에 프로폴리스랑 함께 먹어도 될까요?', '비밀글비밀글입니다.', '제조사가 어떻게 될까요?', '당은 얼마나 함유하고 있나요 ?\n당뇨환자가 먹어도 되나요?', '이거 혀밑에 두고 녹여먹어도되나요?', 'NEW상품구매후 환불요청한 상황입니다!\n상품회수해가신지 일주일이 지났습니다.바로 ㅎ환불처리 해주세요!', '상품 표기 내역에 성분이 많이들어갈수록 앞에 표기되는걸로 알고있는데ㅜㅜ\n콜라겐보다도 정제수랑 블루베리농축액이 더 많이 \n 들어간건가요? \n다른제품들이랑 비교했을때 저거 하나가 걸려서 구매가 망설여지는대 설명 부탁드립니당', '비밀글비밀글입니다.', '비밀글비밀글입니다.', '비밀글비밀글입니다.', '어제 문의글 남겼는데 확인부탁드려요', '임산부 또는 임신 예정인 사람도 먹어도 되나요??', '비밀글비밀글입니다.', '비밀글비밀글입니다.', '7일날 주문했는데 언제발송되나요 ?', '아침 공복에 프로폴리스랑 함께 먹어도 될까요?', '비밀글비밀글입니다.', '제조사가 어떻게 될까요?', '당은 얼마나 함유하고 있나요 ?\n당뇨환자가 먹어도 되나요?', '이거 혀밑에 두고 녹여먹어도되나요?', 'NEW상품구매후 환불요청한 상황입니다!\n상품회수해가신지 일주일이 지났습니다.바로 ㅎ환불처리 해주세요!', '상품 표기 내역에 성분이 많이들어갈수록 앞

In [ ]:
# 파일 경로
file = './rawfile.xlsx'

# 엑셀 파일 불러오기
fileDF = pd.read_excel(file)

for link in fileDF['링크'][:3]:
    # 웹 드라이버 초기화
    agent_option = webdriver.ChromeOptions()
    user_agent_string = 'Mozilla/5.0'
    agent_option.add_argument('user-agent=' + user_agent_string)
    driver = webdriver.Chrome(options=agent_option)

    try:
        # 웹사이트 접속
        driver.get(link)
        driver.implicitly_wait(5)

        # 텍스트를 저장할 리스트 초기화
        text_list = []

        # 페이지 버튼 요소 가져오기
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CLASS_NAME, "UWN4IvaQza._nlog_click")))
        pagenum = driver.find_elements(By.CLASS_NAME, "UWN4IvaQza._nlog_click")

        for i in range(len(pagenum)):
            # 페이지 요소를 매 루프마다 재조회하여 최신화
            driver.implicitly_wait(2)
            pagenum = driver.find_elements(By.CLASS_NAME, "UWN4IvaQza._nlog_click")
            page = pagenum[i]

            # (필요시) 요소로 스크롤 이동
            ActionChains(driver).move_to_element(page).perform()

            # 페이지 버튼 클릭
            driver.implicitly_wait(2)
            page.click()

            # 페이지 HTML 가져오기
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # 텍스트 데이터 추출
            texts = soup.find_all('div', class_='_38rS4qj1Bp')
            for tag in texts:
                name_tag = tag.text.strip()
                text_list.append(name_tag)

        # 결과 출력
        print(text_list)

    finally:
        # 드라이버 종료
        driver.quit()

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF75FE36CB5+28821]
	(No symbol) [0x00007FF75FDA3840]
	(No symbol) [0x00007FF75FC4578A]
	(No symbol) [0x00007FF75FC4C2ED]
	(No symbol) [0x00007FF75FC4EFCC]
	(No symbol) [0x00007FF75FC4F09F]
	(No symbol) [0x00007FF75FC9B34B]
	(No symbol) [0x00007FF75FC8CA96]
	(No symbol) [0x00007FF75FCBF2FA]
	(No symbol) [0x00007FF75FC8C3F6]
	(No symbol) [0x00007FF75FCBF510]
	(No symbol) [0x00007FF75FCDF412]
	(No symbol) [0x00007FF75FCBF0A3]
	(No symbol) [0x00007FF75FC8A778]
	(No symbol) [0x00007FF75FC8B8E1]
	GetHandleVerifier [0x00007FF76016FCAD+3408013]
	GetHandleVerifier [0x00007FF76018741F+3504127]
	GetHandleVerifier [0x00007FF76017B5FD+3455453]
	GetHandleVerifier

- 스마트스토어 qna 불러와서 텍스트 추출하기

In [ ]:
# 사용할 파일
file = './rawfile.xlsx'

# 엑셀 파일 읽기
fileDF = pd.read_excel(file)

# 전체 담을 텍스트 리스트 만들기
text_list = []

for link in fileDF['링크']:
    agent_option = webdriver.ChromeOptions()
    user_agent_string = 'Mozilla/5.0'
    agent_option.add_argument('user-agent=' + user_agent_string)
    driver = webdriver.Chrome(options=agent_option)
    time.sleep(1)

    driver.get(link)
    driver.implicitly_wait(5)
    page_num = driver.find_elements(By.CLASS_NAME, "UWN4IvaQza._nlog_click")
    button = len(page_num)
    driver.quit()
    time.sleep(1)

    ### 페이지 루프 돌면서 텍스트 가져오기
    for i in range(1, button+1):
        try:
            agent_option = webdriver.ChromeOptions()
            user_agent_string = 'Mozilla/5.0'
            agent_option.add_argument('user-agent=' + user_agent_string)
            driver = webdriver.Chrome(options=agent_option)

            target = link + f'?filterMyComment=false&page={i}'
            driver.get(target)
            time.sleep(1)

            # 페이지별 텍스트 추출 리스트
            text_in_page = []
            
            # HTML 가져오기
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            # 필요한 데이터 추출
            texts = [tag.text.strip() for tag in soup.find_all('div', class_='_38rS4qj1Bp')]
            text_in_page.extend(texts)
            text_list.append(text_in_page)
            driver.quit()
            time.sleep(1)
            
        # 오류 발생 시 예외 처리 후 넘어감.
        except Exception as e:
            print(f'예외 오류 발생 코드: {e}')
            continue
            

    # 결과 저장
    textDF = pd.DataFrame({'텍스트': text_list})
    textDF.to_excel(textfile, index=False)